In [3]:
import numpy as np
import pandas as pd
import re
import string
from nltk import word_tokenize
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import nltk

In [4]:
data = pd.read_csv("all_igbo_news_data.csv", index_col = False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      833 non-null    object
 1   category  833 non-null    object
 2   source    833 non-null    object
dtypes: object(3)
memory usage: 19.7+ KB


In [5]:
data.head()

,text,category,source
0,Ụfọdụ ihe i kwesịrị ịma gbasara 'Chairman' Chr...,Egwuruegwu,https://www.bbc.com/igbo/articles/c793q1dql32o
1,Mmasị m nwere n'ịbụ onyendu mere m jiri malite...,Egwuruegwu,https://www.bbc.com/igbo/articles/cdxq4yw4117o
2,Ademola Lookman bụ ọkacha agba bọọlụ ebe Chiam...,Egwuruegwu,https://www.bbc.com/igbo/articles/c4gldjvl1e3o
3,Agamnihu m nwetara n’ihe karịrị afọ abụọ bụ nn...,Egwuruegwu,https://www.bbc.com/igbo/articles/cwy5gyp5qd8o
4,NEC agwala Onyeisiala Tinubu ka o wezuga atụma...,Egwuruegwu,https://www.bbc.com/igbo/live/cx2m781lrrmt


In [6]:
data["category"].value_counts()

,count
category,
Egwuruegwu,424
Ihe nkiri,341
ochichi,10
nchekwa,10
akuko,10
mmepe-obodo,7
mmemme,5
ahu-ike,4
omenaala,3


In [7]:
text = data["text"].sample(50)
text

,text
585,Ọrịa 'gigantism' mere ka m bụrụ onye kachasị o...
443,Mmasị m nwere n'ịbụ onyendu mere m jiri malite...
2,Ademola Lookman bụ ọkacha agba bọọlụ ebe Chiam...
639,"Peter Obi: Ihe ndị ị maghị maka Obi, Tinubu, P..."
571,Miss Universe 2023: Ọ bụghị naanị mma ihu na m...
731,Hair and Nail Designer: Ihe m ji bụrụ ọkaibe n...
673,Chef Mohammed: Etu m si malite isi nri ọdịnala...
456,A kpụpụla nwoke bepụrụ nwunye ya aka abụọ ụlọi...
600,A mụtara m asụsụ ndị Chịana n’Anambra tupu m a...
50,Qatar world cup: Nchikọ Akụkọ Iko Mbaụwa Pụrụi...


In [8]:
for i, row in text.items():
  t = row.split()
  print(t)

['Ọrịa', "'gigantism'", 'mere', 'ka', 'm', 'bụrụ', 'onye', 'kachasị', 'ogologo', 'na', 'Ghana', '–', 'Sulemana', 'Abdul', 'LIVE', 'START', 'LIVE', 'Video', 'quality', 'Low', 'Medium', 'Highest', 'available', 'Automatically', 'selects', 'the', 'best', 'quality', 'available', '00:00', '03:46', 'Nwaa', 'ọzọ', 'Aha', 'onyonyo,', 'A', 'bụ', 'm', 'nwoke', 'kachaa', 'too', 'ogologo', 'na', 'Gana', '-', 'Sulemana', 'Abdul', 'Samed', 'Ọrịa', "'gigantism'", 'mere', 'ka', 'm', 'bụrụ', 'onye', 'kachasị', 'ogologo', 'na', 'Ghana', '–', 'Sulemana', 'Abdul', '30', 'Ọktọba', '2022', 'Mgbe', 'ikpeazụ', 'e', 'tinyere', 'ya', 'ozi', 'ọhụrụ', '3', 'Ọgọọst', '2023', 'Nwafọ', 'mba', 'Ghana', 'bụ', 'Suleman', 'Abdul', 'Samed,', 'a', 'maara', 'dịka', 'Awuche', 'bụ', 'nwoke', 'kachasị', 'too', 'ogologo', 'na', 'mba', 'ha.', 'Ọ\ufeff', 'kọwara', 'etu', 'ọrịa', 'ito', 'ogologo', 'a', 'kpọrọ', "'gigantism'", 'jiri', 'mee', 'ka', 'ọ', 'bụrụ', 'onye', 'kachasị', 'ogologo', 'na', 'mba', 'Ghana.', 'A\ufeffbdul', 'sị'

Splitting done above shows some issues with the text that needs to be fixed.
1. Remove punctuations and special characters.
2. Normalise case to lower case.
3. Remove stopwords like ("na", "a", "ya", "ha", etc.).


In [9]:
data.head()

,text,category,source
0,Ụfọdụ ihe i kwesịrị ịma gbasara 'Chairman' Chr...,Egwuruegwu,https://www.bbc.com/igbo/articles/c793q1dql32o
1,Mmasị m nwere n'ịbụ onyendu mere m jiri malite...,Egwuruegwu,https://www.bbc.com/igbo/articles/cdxq4yw4117o
2,Ademola Lookman bụ ọkacha agba bọọlụ ebe Chiam...,Egwuruegwu,https://www.bbc.com/igbo/articles/c4gldjvl1e3o
3,Agamnihu m nwetara n’ihe karịrị afọ abụọ bụ nn...,Egwuruegwu,https://www.bbc.com/igbo/articles/cwy5gyp5qd8o
4,NEC agwala Onyeisiala Tinubu ka o wezuga atụma...,Egwuruegwu,https://www.bbc.com/igbo/live/cx2m781lrrmt


In [11]:
def clean(data):
  #split words by whitespace
  data["clean_text"] = data["text"].apply(lambda x: x.split())

  # save the pattern for the punctuation and special characters in regex
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))

  # replace the punctuation with nothing for each word in the list
  # and join the words back into a string
  data["clean_text"] = data["clean_text"].apply(lambda x: ' '.join([re_punc.sub('', word) for word in x]))

  # remove extra special characters not handled above
  re_s = re.compile('[’,-]')
  data["clean_text"] = data["clean_text"].apply(lambda x: ' '.join([re_s.sub('', word) for word in x.split()]))

  # remove extra whitspace
  data["clean_text"] = data["clean_text"].apply(lambda x: x.strip())

  # lower case
  data["clean_text"] = data["clean_text"].apply(lambda x: x.lower())

  # find number of tokens and unique tokens
  unique_tokens = set()
  tokens = data["clean_text"].apply(lambda x: x.split())
  count = 0
  for i, row in tokens.items():
    for word in row:
      count += 1
      unique_tokens.add(word)

  print("Data successfully cleaned!")
  print("The total number of tokens = " + str(count))


  return data["clean_text"]

In [12]:
data["clean_text"] = clean(data)
data["clean_text"]

Data successfully cleaned!
The total number of tokens = 435351


,clean_text
0,ụfọdụ ihe i kwesịrị ịma gbasara chairman chris...
1,mmasị m nwere nịbụ onyendu mere m jiri malite ...
2,ademola lookman bụ ọkacha agba bọọlụ ebe chiam...
3,agamnihu m nwetara nihe karịrị afọ abụọ bụ nnọ...
4,nec agwala onyeisiala tinubu ka o wezuga atụma...
...,...
828,akuko naewu anambra state library board flags ...
829,akuko naewu anambra state library board flags ...
830,akuko naewu anambra state library board flags ...
831,akuko naewu anambra state library board flags ...


In [13]:
# Initialize a defaultdict with integer values to store word frequencies
vocab = defaultdict(int)

corpus = data["clean_text"]

# Loop through each sentence in the corpus and count no. of occurrence of each word
for article in corpus.apply(lambda x: x.split()):
    for word in article:
        vocab[word] += 1

# Convert the defaultdict vocab to a regular dictionary for easier handling and sorting
# and sort the dictionary by word frequency in descending order
sorted_vocab = dict(sorted(vocab.items(), key=lambda x: x[1], reverse=True))

# Display the sorted vocabulary with each word and its frequency count
print("Vocabulary with Frequencies:", sorted_vocab)

Vocabulary with Frequencies: {'na': 20558, 'ndị': 11015, 'ya': 9513, 'a': 7504, 'bụ': 7495, 'ha': 7282, 'ihe': 5841, 'nke': 5718, 'ọ': 5657, 'ka': 4724, 'ahụ': 4194, 'ebe': 4052, 'si': 3986, 'onye': 3606, 'dị': 3395, 'foto': 3308, 'otu': 3140, 'ma': 2709, 'o': 2709, 'dịka': 2484, 'maka': 2453, 'mba': 2401, 'm': 2329, 'e': 2189, 'ọzọ': 2173, 'kwuru': 2060, 'mmadụ': 1969, 'mere': 1962, 'naịjirịa': 1891, 'anyị': 1888, 'ji': 1672, 'steeti': 1651, 'x': 1643, 'akụkọ': 1601, 'bọọlụ': 1489, 'ọrụ': 1479, 'afọ': 1475, 'nwere': 1457, 'aka': 1373, 'aha': 1362, 'abụọ': 1269, 'nime': 1239, 'mana': 1217, 'ị': 1194, 'nkọwa': 1176, 'iji': 1168, 'iwu': 1155, 'tupu': 1150, 'nihu': 1146, 'etu': 1127, 'nabata': 1116, 'mgbe': 1082, 'nọ': 1081, 'igbo': 1026, 'gaa': 995, 'sị': 980, 'images': 971, 'iri': 959, 'getty': 957, 'ọbụla': 952, 'bbc': 947, 'niile': 935, 'nwereike': 922, '2': 895, 'i': 886, 'anya': 868, 'onyonyo': 843, 'egwu': 819, 'ugbua': 812, 'nkeji': 811, 'post': 807, 'ọtụtụ': 798, 'oge': 797, 'ego

In [38]:
IGBO_STOPWORDS = {
    "ihe",
    "na",
    "nke",
    "mana",
    "n’",
    "ka",
    "ma",
    "ya",
    "si",
    "ga",
    "di",
    "onye",
    "ndị",
    "bụ",
    "ọ",
    "a",
    "e",
    "i",
    "ị",
    "o",
    "ha",
    "unu",
    "anyị",
    "gị",
    "m",
    "mụ",
    "ọ bụla",
    "ọbụna",
    "gịnị",
    "ebee",
    "olee",
    "ole",
    "ahụ",
    "kedụ",
    "maka",
    "mụ onwe m",
    "onwe m",
    "gị onwe gị",
    "onwe gị",
    "ya onwe ya",
    "onwe ya",
    "anyị onwe anyị",
    "onwe anyị",
    "unu onwe unu",
    "onwe unu",
    "ha onwe ha",
    "onwe ha",
    "mụ wa",
    "gị wa",
    "ya wa",
    "anyị wa",
    "unu wa",
    "ha wa",
    "mụ nwa",
    "gị nwa",
    "ya nwa",
    "anyị nwa",
    "unu nwa",
    "ha nwa",
    "na",
    "n’",
    "n'",
    "n\"",
    "n’ime",
    "nime",
    "niime",
    "n’elu",
    "nelu",
    "neelu",
    "n’ihi",
    "niihi",
    "nihi",
    "naka",
    "naaka",
    "n’aka",
    "nakụkụ",
    "naakụkụ",
    "n’akụkụ",
    "n’okpuru",
    "nokpuru",
    "noookpuru",
    "n’ebe",
    "nebe",
    "nebee",
    "n’ebee",
    "n’anya",
    "nanya",
    "n’ụzọ",
    "nụzọ",
    "na nkọnkọ",
    "n’azụ",
    "nazụ",
    "ganye ma",
    "ganye na",
    "ya mere",
    "gbasara",
    "site",
    "mgbe",
    "n’ezie",
    "nezie",
    "kama",
    "ka ọ bụ",
    "kaọbụ",
    "ka ọ bụrụ",
    "kaọbụrụ",
    "ma ọ bụ",
    "maọbụ",
    "ma ọ bụrụ",
    "maọbụrụ",
    "kamgbe",
    "kemgbe",
    "nwere ike",
    "mee",
    "gunyere",
    "banyere"
    "niile",
    "nile",
    "ọtụtụ",
    "ụfọdụ",
    "ka o si na dị",
    "ka o si dị",
    "ka ọ na dị",
    "maka - because",
    "kwa",
    "mana",
    "maọbụ",
    "tupu",
    "tutuu",
    "dị",
    "dịka",
    "dịghị",
    "ghị",
    "beghi",
    "bụghị",
    "gaghị",
    "ee",
    "mba",
    "ihe",
    "ruo",
    "rute"}


In [39]:
for word, freq in sorted_vocab.items():
  if freq >= 3000:
    IGBO_STOPWORDS.add(word)
print(IGBO_STOPWORDS)
print(len(IGBO_STOPWORDS))

{'kama', 'site', 'na', 'bụ', 'ka ọ bụ', 'n’ime', 'olee', 'n’', 'ha nwa', 'ọtụtụ', 'm', 'ya mere', 'gị', 'foto', 'naaka', 'dị', 'gbasara', 'ahụ', 'o', 'tutuu', 'gunyere', 'mụ onwe m', 'unu', 'mana', 'nebee', 'unu nwa', 'naka', 'n’aka', 'kaọbụ', 'banyereniile', 'ka', 'ị', 'otu', 'ka o si na dị', "n'", 'si', 'ka o si dị', 'ganye na', 'ndị', 'n’elu', 'maọbụ', 'kedụ', 'ee', 'nile', 'ya nwa', 'onwe gị', 'nwere ike', 'gị nwa', 'maka - because', 'beghi', 'anyị nwa', 'ha', 'n’anya', 'mụ wa', 'onye', 'unu onwe unu', 'kamgbe', 'gị onwe gị', 'ihe', 'di', 'anyị', 'mụ', 'ha onwe ha', 'ya', 'dịghị', 'ebe', 'kemgbe', 'ma ọ bụrụ', 'n’ebee', 'n’ụzọ', 'ya onwe ya', 'ole', 'nokpuru', 'ya wa', 'ruo', 'maka', 'onwe anyị', 'dịka', 'nebe', 'n"', 'ka ọ na dị', 'unu wa', 'ghị', 'gịnị', 'ọ bụla', 'nakụkụ', 'n’ezie', 'ọ', 'anyị wa', 'niihi', 'noookpuru', 'gaghị', 'ọbụna', 'kaọbụrụ', 'na nkọnkọ', 'kwa', 'naakụkụ', 'nime', 'ha wa', 'ka ọ bụrụ', 'ma ọ bụ', 'bụghị', 'gị wa', 'nanya', 'n’azụ', 'anyị onwe anyị', 'niime

In [41]:
# extra words to clean out from scraping
i = ["live","getty","getty fotos", "images","fotos","Video","quality","Low","Medium","Highest","available","Automatically","selects","the","best","quality","available","Play","afp"]
IGBO_STOPWORDS.update(i)
print(IGBO_STOPWORDS)
print(len(IGBO_STOPWORDS))

{'kama', 'site', 'na', 'bụ', 'ka ọ bụ', 'n’ime', 'olee', 'n’', 'ha nwa', 'ọtụtụ', 'm', 'ya mere', 'gị', 'foto', 'naaka', 'dị', 'gbasara', 'ahụ', 'best', 'o', 'tutuu', 'gunyere', 'mụ onwe m', 'unu', 'mana', 'nebee', 'unu nwa', 'naka', 'n’aka', 'kaọbụ', 'banyereniile', 'ka', 'ị', 'otu', 'ka o si na dị', "n'", 'the', 'si', 'ka o si dị', 'ganye na', 'ndị', 'n’elu', 'maọbụ', 'kedụ', 'ee', 'nile', 'ya nwa', 'onwe gị', 'nwere ike', 'gị nwa', 'maka - because', 'beghi', 'anyị nwa', 'ha', 'n’anya', 'mụ wa', 'onye', 'unu onwe unu', 'kamgbe', 'Automatically', 'gị onwe gị', 'ihe', 'di', 'selects', 'anyị', 'mụ', 'ha onwe ha', 'ya', 'dịghị', 'ebe', 'quality', 'kemgbe', 'ma ọ bụrụ', 'images', 'n’ebee', 'n’ụzọ', 'Video', 'ya onwe ya', 'ole', 'nokpuru', 'ya wa', 'ruo', 'maka', 'onwe anyị', 'getty', 'dịka', 'nebe', 'n"', 'ka ọ na dị', 'unu wa', 'ghị', 'gịnị', 'ọ bụla', 'nakụkụ', 'n’ezie', 'ọ', 'anyị wa', 'niihi', 'noookpuru', 'gaghị', 'ọbụna', 'kaọbụrụ', 'na nkọnkọ', 'kwa', 'naakụkụ', 'nime', 'ha wa', 'k

In [42]:
def remove_stopwords(text, stopwords):
    words = text.split()  # tokenize by whitespace
    filtered_words = [word for word in words if word not in stopwords]
    return " ".join(filtered_words)

# Apply the function to each article in the DataFrame
data["clean_text"] = data["clean_text"].apply(lambda article: remove_stopwords(article, IGBO_STOPWORDS))

data.head()

,text,category,source,clean_text
0,Ụfọdụ ihe i kwesịrị ịma gbasara 'Chairman' Chr...,Egwuruegwu,https://www.bbc.com/igbo/articles/c793q1dql32o,kwesịrị ịma chairman christian chukwu 12 eprel...
1,Mmasị m nwere n'ịbụ onyendu mere m jiri malite...,Egwuruegwu,https://www.bbc.com/igbo/articles/cdxq4yw4117o,mmasị nwere nịbụ onyendu mere jiri malite ịchị...
2,Ademola Lookman bụ ọkacha agba bọọlụ ebe Chiam...,Egwuruegwu,https://www.bbc.com/igbo/articles/c4gldjvl1e3o,ademola lookman ọkacha agba bọọlụ chiamaka nna...
3,Agamnihu m nwetara n’ihe karịrị afọ abụọ bụ nn...,Egwuruegwu,https://www.bbc.com/igbo/articles/cwy5gyp5qd8o,agamnihu nwetara nihe karịrị afọ abụọ nnọọ ezi...
4,NEC agwala Onyeisiala Tinubu ka o wezuga atụma...,Egwuruegwu,https://www.bbc.com/igbo/live/cx2m781lrrmt,nec agwala onyeisiala tinubu wezuga atụmatụ in...


In [43]:
data.to_csv("igbo_data_clean.csv", index = False)